## 掛載雲端硬碟

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##更改檔案所在路徑


In [2]:
# Change to your own folder !!!
%cd /content/drive/MyDrive/EAI_Lab4_2024

/content/drive/MyDrive/EAI_Lab4_2024


## 載入函式庫


In [3]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np

from models import vgg

##超參數設定

In [4]:
DATASET = 'cifar10'
TEST_BATCH_SIZE = 1000
CUDA = True
PRUNE_PERCENT = 0.9 # Change your prune ratio!
WEIGHT_PATH = '/content/drive/MyDrive/Colab Notebooks/model_best.pth'
PRUNE_PATH = '/content/drive/MyDrive/Colab Notebooks/model_prune.pth'


##載入模型

In [5]:
CUDA = CUDA and torch.cuda.is_available()

model = vgg()
if CUDA:
    model.cuda()

if WEIGHT_PATH:
    if os.path.isfile(WEIGHT_PATH):
        checkpoint = torch.load(WEIGHT_PATH)
        best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        print('LOADING CHECKPOINT {} @EPOCH={}, BEST_PREC1={}'.format(WEIGHT_PATH,checkpoint['epoch'],best_prec1))

    else:
        print("NO CHECKPOINT FOUND")

print(model)

<ipython-input-5-d2659840b6c9>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(WEIGHT_PATH)


LOADING CHECKPOINT /content/drive/MyDrive/Colab Notebooks/model_best.pth @EPOCH=46, BEST_PREC1=0.8823000192642212
vgg(
  (feature): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, trac

##測試函數(觀察模型精確度)


In [6]:
def test(model):
    kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {}
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])),
        batch_size=TEST_BATCH_SIZE, shuffle=True, **kwargs)
    model.eval()
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
          if CUDA:
              data, target = data.cuda(), target.cuda()
          data, target = Variable(data), Variable(target)
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1]
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.1f}%)\n'.format(
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))

## 進行剪枝
#### 計算所有Batch Normalizaiton中的scale factor絕對值大小並排序
#### 利用PRUNE_RATIO中取得閥值

In [7]:
total = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        total += m.weight.data.shape[0]

bn = torch.zeros(total)
index = 0
for m in model.modules():
    if isinstance(m, nn.BatchNorm2d):
        size = m.weight.data.shape[0]
        bn[index:(index+size)] = m.weight.data.abs().clone()
        index += size

y, i = torch.sort(bn)


threshold_index = int(total * PRUNE_PERCENT)
threshold = y[threshold_index]


##建立CONFIG，之後建立剪枝後網路時需要用到此CONFIG

In [8]:
pruned = 0
cfg = []  #用來建立剪枝網路的CONFIG
cfg_mask = [] #用來幫助剪枝的遮罩

##根據Batch Normalization Layer資訊建立CONFIG
####1.複製Batch Normalization Layer的weight(也就是scale factor)
####2.建立mask，大於threshold的index的值會設成1,小於threshold的值會設成0
####3.大於threshold的index的值加總後，會是剪枝後Layer對應的輸出channel
####4.最後得到要建立剪枝模型的CONFIG

In [9]:
for k, m in enumerate(model.modules()):
    if isinstance(m, nn.BatchNorm2d):
        weight_copy = m.weight.data.clone()
        mask = weight_copy.abs().gt(threshold).float().cuda()

        # 注意: 需自行設計處理剩下channel數為0的情況 (e.g. 至少保留3個channel)
        if isinstance(m, nn.BatchNorm2d):
          # 1. 複製 Batch Normalization Layer 的 weight
          weight_copy = m.weight.data.clone()

          # 2. 建立 mask，大於 threshold 的設為 1，小於 threshold 的設為 0
          mask = weight_copy.abs().gt(threshold).float().cuda()

        # 確保每層至少保留 3 個通道
        if torch.sum(mask) < 3:
            _, top_indices = torch.topk(weight_copy.abs(), 3)  # 取得絕對值最大的前 3 個通道的索引
            mask[top_indices] = 1.0  # 保留這 3 個通道

        # 計算剪枝的通道數量並更新總計
        pruned = pruned + mask.shape[0] - torch.sum(mask)
        cfg.append(int(torch.sum(mask)))
        cfg_mask.append(mask.clone())
        print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.
            format(k, mask.shape[0], int(torch.sum(mask))))
    elif isinstance(m, nn.MaxPool2d):
        cfg.append('M')

pruned_ratio = pruned/total

print(f'PRUNE RATIO={pruned_ratio}')
print('PREPROCESSING SUCCESSFUL!')

print(cfg)


layer index: 3 	 total channel: 64 	 remaining channel: 21
layer index: 6 	 total channel: 64 	 remaining channel: 43
layer index: 10 	 total channel: 128 	 remaining channel: 61
layer index: 13 	 total channel: 128 	 remaining channel: 66
layer index: 17 	 total channel: 256 	 remaining channel: 81
layer index: 20 	 total channel: 256 	 remaining channel: 56
layer index: 23 	 total channel: 256 	 remaining channel: 43
layer index: 26 	 total channel: 256 	 remaining channel: 38
layer index: 30 	 total channel: 512 	 remaining channel: 26
layer index: 33 	 total channel: 512 	 remaining channel: 17
layer index: 36 	 total channel: 512 	 remaining channel: 15
layer index: 39 	 total channel: 512 	 remaining channel: 22
layer index: 43 	 total channel: 512 	 remaining channel: 12
layer index: 46 	 total channel: 512 	 remaining channel: 12
layer index: 49 	 total channel: 512 	 remaining channel: 19
layer index: 52 	 total channel: 512 	 remaining channel: 18
PRUNE RATIO=0.90007269382476

###建立剪枝模型

In [10]:
newmodel = vgg(cfg=cfg)
newmodel.cuda()

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(21, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(21, 43, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(43, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(43, 61, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(61, 66, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(66, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

###將原本的模型權重複製到剪枝的模型
####1.決定該層的輸入與輸出Channel
####2.根據不同層決定要複製什麼權重
######Batch Normalization Layer
1.   scale factor
2.   bias
3.   running mean
4.   running variance

######Convolutional Layer
1.   weight
2.   bias

######Linear Layer
1.   weight
2.   bias



In [11]:
layer_id_in_cfg = 0
start_mask = torch.ones(3) #3為input channel(R,G,B)
end_mask = cfg_mask[layer_id_in_cfg]
count = 0
for [m0, m1] in zip(model.modules(), newmodel.modules()):
    if isinstance(m0, nn.BatchNorm2d):

        # 處理剪枝後的權重
        m0.weight.data.mul_(end_mask)
        m0.bias.data.mul_(end_mask)

        #### 找出遮罩中非零元素的index ####
        ################################################
        #          請填空          #
        idx = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx.size == 1:
            idx = np.resize(idx, (1,))  # 確保索引維度一致
        ################################################


        # 將原本模型的權重複製到剪枝模型的權重

        #### 複製weight與bias ####
        ################################################
        #          請填空          #
        m1.weight.data = m0.weight.data[idx.tolist()].clone()
        m1.bias.data = m0.bias.data[idx.tolist()].clone()

        ################################################


        #### 複製running mean跟running variance ####
        ################################################
        #          請填空          #
        m1.running_mean = m0.running_mean[idx.tolist()].clone()
        m1.running_var = m0.running_var[idx.tolist()].clone()

        ################################################

        layer_id_in_cfg += 1
        start_mask = end_mask.clone()

        #最後一層連接層不做修改
        if layer_id_in_cfg < len(cfg_mask):
            end_mask = cfg_mask[layer_id_in_cfg]
    elif isinstance(m0, nn.Conv2d):
        # 將原本模型的捲積層權重複製到對應剪枝模型卷積層的權重
        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))

        w = m0.weight.data[:, idx0, :, :].clone()
        w = w[idx1, :, :, :].clone()
        m1.weight.data = w.clone()
        #m1.bias.data = m0.bias.data[idx1].clone()
    elif isinstance(m0, nn.Linear):
        # 參考 https://pytorch.org/docs/stable/generated/torch.nn.Linear.html 來決定該如何複製Linear Layer參數

        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        if idx0.size == 1:
            idx0 = np.resize(idx0, (1,))

        #### 複製weight ####
        ################################################
        #          請填空          #
        m1.weight.data = m0.weight.data[:, idx0.tolist()].clone()
        ################################################



        #### 複製bias ####
        ################################################
        #          請填空          #
        m1.bias.data = m0.bias.data.clone()

        ################################################



####儲存模型並印出結果


In [12]:
torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()}, PRUNE_PATH)

print(newmodel)
model = newmodel
test(newmodel)

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(21, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(21, 43, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(43, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(43, 61, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(61, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(61, 66, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(66, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, 

tensor(0.2615)